# AI-Powered HR Assistant for Nestlé's HR Policy

This notebook demonstrates how to build a conversational chatbot that can answer questions about Nestlé's Human Resources policies using a PDF document as its knowledge base. The project uses the [LangChain](https://python.langchain.com) library to load and process the PDF, create embeddings, perform retrieval using a vector store, and interact with OpenAI's GPT model for generating answers. A simple user interface is built with [Gradio](https://gradio.app) to allow interactive querying.

**Note:** You need an OpenAI API key with access to the `gpt-3.5-turbo` or higher model to run this notebook. Replace `'YOUR-OPENAI-API-KEY'` with your actual API key in the setup section.


## 1. Setup environment and load dependencies

In this section, we import necessary libraries and configure the OpenAI API. We also load the Nestlé HR policy PDF using LangChain's `PyPDFLoader` and split it into manageable text chunks.


In [ ]:

# Install dependencies (if needed) - uncomment the next lines on first run
# !pip install langchain==0.0.353 chromadb==0.4.2 openai==1.6.1 gradio==4.8.0 tiktoken==0.5.1

import os

# Set your OpenAI API key
import openai
openai.api_key = os.getenv("OPENAI_API_KEY", "YOUR-OPENAI-API-KEY")

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

# Path to the Nestlé HR policy PDF
pdf_path = "/home/oai/share/1728286846_the_nestle_hr_policy_pdf_2012.pdf"

# Load and parse the PDF
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# Split documents into chunks for efficient embedding and retrieval
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

texts = text_splitter.split_documents(documents)

# Create embeddings for the text chunks
embeddings = OpenAIEmbeddings()

# Initialize Chroma vector store to store and search over embeddings
vector_store = Chroma.from_documents(texts, embeddings)

# Create a conversational retrieval QA chain using GPT-3.5 Turbo model
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

qa_chain = ConversationalRetrievalChain.from_llm(llm, vector_store.as_retriever())



## 2. Define the chat function

Here we define a function that will interact with the `qa_chain`. It accepts a user question and a conversation history, sends the question to the chain, and returns the answer. The history is important for maintaining the conversation context.


In [ ]:

# Conversation history will be stored as a list of (question, answer) tuples

def ask_chatbot(question, history=[]):
    """Get an answer from the chatbot given a question and history."""
    # Format history for the conversational retrieval chain
    chat_history = [(entry[0], entry[1]) for entry in history]
    result = qa_chain({"question": question, "chat_history": chat_history})
    answer = result["answer"]
    return answer



## 3. Build the Gradio Interface

Using Gradio, we create a simple web interface that allows a user to type a question and receive an answer from the HR assistant. The interface maintains the chat history automatically.


In [ ]:

import gradio as gr

# Define the Gradio chatbot function
def gradio_chatbot(user_input, chat_history):
    # Convert Gradio chat history format [(user, bot), ...] to our format
    formatted_history = [(entry[0], entry[1]) for entry in chat_history or []]
    answer = ask_chatbot(user_input, formatted_history)
    chat_history = chat_history or []
    chat_history.append((user_input, answer))
    return "", chat_history

# Create the Gradio blocks interface
with gr.Blocks(title="Nestlé HR Assistant") as demo:
    gr.Markdown("""### Nestlé HR Policy Chatbot
Ask a question about the Nestlé HR policy document and get answers powered by GPT and document retrieval.""")
    chatbot = gr.Chatbot(label="Chat History")
    message = gr.Textbox(label="Your question", placeholder="Type your HR-related question here...")
    clear_btn = gr.Button("Clear Chat")
    
    def submit_message(user_input, chat_history):
        return gradio_chatbot(user_input, chat_history)

    message.submit(submit_message, [message, chatbot], [message, chatbot])
    clear_btn.click(lambda: ("", []), None, [message, chatbot], queue=False)

demo
